In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/PhoneLog/raw_phonelog_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')


In [4]:
df = df.dropna()
print(len(df))

3349


In [5]:
df.columns

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')

In [6]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    1328
high stress      1043
low stress        978
Name: count, dtype: int64

In [7]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [8]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

logo = LeaveOneGroupOut()

In [9]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [10]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [11]:
num_splits = len(np.unique(groups))
print(num_splits)

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print('X and y train shapes: ')
        print(X_train.shape)
        print(y_train.shape)

        train_class_counts = np.bincount(y_train)
        print(f"Train set - Class 0 (Low Stress): {train_class_counts[0]}, "
              f"Class 1 (High Stress): {train_class_counts[1] if len(train_class_counts) > 1 else 0}")

        test_class_counts = np.bincount(y_test)
        print(f"Test set - Class 0 (Low Stress): {test_class_counts[0]}, "
              f"Class 1 (High Stress): {test_class_counts[1] if len(test_class_counts) > 1 else 0}")

        print(f"Test set size: {len(y_test)}")
        print(f"Train set shape: {X_train.shape}")
        print(f"Test set shape: {X_test.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(X_train_normalized.shape)
        print(X_test_normalized.shape)

        # Define the FCN model
        model_fcn_binary_lh = tf.keras.models.Sequential([
            tf.keras.layers.Dense(57, activation='relu', input_dim=X_train_normalized.shape[1]),
            tf.keras.layers.Dropout(0.35),
            tf.keras.layers.Dense(35, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(3, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
        ])
        print('Model Defined')

        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model_fcn_binary_lh.compile(
            optimizer=custom_optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']  
        )

        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(enumerate(class_weights))

        print('Class Weight: ', class_weight_dict)

        model_fcn_binary_lh.fit(
            X_train_normalized, y_train,
            epochs=100,
            batch_size=32,
            class_weight=class_weight_dict,
            verbose=1  
        )

        thresholds = np.arange(0.01, 1.0, 0.01)
        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        best_threshold = 0.0
        best_metric = 0.0

        for threshold in thresholds:
            y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
            metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)

            if metric_value > best_metric:
                best_metric = metric_value
                best_threshold = threshold

        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)  
            if auc_score < 0.5:
                print(f"<0.5 AUC Score: {auc_score}")
                print("----------------------------------------")
            else:
                print(f"AUC Score: {auc_score}")

        else:
            auc_scores.append(None)  
            print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc) 
        best_thresholds.append(best_threshold)  
        print(f"Balanced Accuracy: {balanced_acc}")

        pbar.update(1)

43


LOSO CV Progress:   0%|                                    | 0/43 [00:00<?]   0%

X and y train shapes: 
(1950, 20)
(1950,)
Train set - Class 0 (Low Stress): 948, Class 1 (High Stress): 1002
Test set - Class 0 (Low Stress): 30, Class 1 (High Stress): 41
Test set size: 71
Train set shape: (1950, 20)
Test set shape: (71, 20)
(1950, 20)
(71, 20)


Model Defined
Class Weight:  {0: 1.0284810126582278, 1: 0.9730538922155688}
Epoch 1/100


2024-10-05 10:56:26.530665: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


...

AUC Score: 0.541445623342175
Balanced Accuracy: 0.5431034482758621
X and y train shapes: 
(1939, 20)
(1939,)
Train set - Class 0 (Low Stress): 961, Class 1 (High Stress): 978
Test set - Class 0 (Low Stress): 17, Class 1 (High Stress): 65
Test set size: 82
Train set shape: (1939, 20)
Test set shape: (82, 20)
(1939, 20)
(82, 20)
Model Defined
Class Weight:  {0: 1.0088449531737773, 1: 0.9913087934560327}
Epoch 1/100
61/61 [==============================] - 0s 474us/step - loss: 0.7085 - accuracy: 0.5018
Epoch 2/100
61/61 [==============================] - 0s 419us/step - loss: 0.6967 - accuracy: 0.4992
Epoch 3/100
61/61 [==============================] - 0s 421us/step - loss: 0.6933 - accuracy: 0.4982
Epoch 4/100
61/61 [==============================] - 0s 415us/step - loss: 0.6930 - accuracy: 0.4982
Epoch 5/100
61/61 [==============================] - 0s 412us/step - loss: 0.6932 - accuracy: 0.5044
Epoch 6/100
61/61 [==============================] - 0s 411us/step - loss: 0.6930 - accura

LOSO CV Progress:  98%|█████████████████████████████▎ | 42/43 [05:19<00:04]  98%WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


AUC Score: 0.5429864253393665
Balanced Accuracy: 0.5552036199095023
X and y train shapes: 
(1869, 20)
(1869,)
Train set - Class 0 (Low Stress): 858, Class 1 (High Stress): 1011
Test set - Class 0 (Low Stress): 120, Class 1 (High Stress): 32
Test set size: 152
Train set shape: (1869, 20)
Test set shape: (152, 20)
(1869, 20)
(152, 20)
Model Defined
Class Weight:  {0: 1.0891608391608392, 1: 0.9243323442136498}
Epoch 1/100
59/59 [==============================] - 0s 431us/step - loss: 0.6998 - accuracy: 0.4880
Epoch 2/100
59/59 [==============================] - 0s 404us/step - loss: 0.6919 - accuracy: 0.5029
Epoch 3/100
59/59 [==============================] - 0s 419us/step - loss: 0.6956 - accuracy: 0.5238
Epoch 4/100
59/59 [==============================] - 0s 408us/step - loss: 0.6942 - accuracy: 0.4703
Epoch 5/100
59/59 [==============================] - 0s 402us/step - loss: 0.6924 - accuracy: 0.4746
Epoch 6/100
59/59 [==============================] - 0s 404us/step - loss: 0.6929 - 

LOSO CV Progress: 100%|██████████████████████████████ | 43/43 [05:22<00:00] 100%

<0.5 AUC Score: 0.49635416666666665
----------------------------------------
Balanced Accuracy: 0.5541666666666667


In [12]:
print('MLP: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

MLP: 
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.5171957671957672, 25th Percentile: 0.4297339188840093, 75th Percentile: 0.572787488201022
Balanced Accuracy - Median: 0.564957264957265, 25th Percentile: 0.5267094017094017, 75th Percentile: 0.6597222222222222
